In [1]:
import os
import pandas as pd
import numpy as np
import requests
import json
import matplotlib.pyplot as plt
import seaborn as sns
import gmaps
import gmaps.geojson_geometries
import time
google_maps_api_key = os.environ.get('googlemaps_api_key')
gmaps.configure(api_key=google_maps_api_key)

In [2]:
df = pd.read_csv('raw_data\\neighborhood_commute_and_transit_times.csv')
df.drop('Unnamed: 0', axis='columns', inplace=True)
df.head()

,Neighborhood,House Value,Median Rent,POPULATION_2010,PCT_WHITE,DENVER_REGION,drive-AM-to-Downtown,transit-AM-to-Downtown,drive-PM-from-Downtown,transit-PM-from-Downtown,...,% Slower Rush Hour Drive AM-to-DTC,Transit % Slower AM-to-DTC,% Slower Rush Hour Drive PM-from-DTC,Transit % Slower PM-from-DTC,% Slower Rush Hour Drive AM-to-CU-Boulder,Transit % Slower AM-to-CU-Boulder,% Slower Rush Hour Drive PM-from-CU-Boulder,Transit % Slower PM-from-CU-Boulder,Average % Slower Rush Hour Drive,Average Transit % Slower
0,Montbello,174000.0,1854.0,30348,8.17,Northeast,{'geocoded_waypoints': [{'geocoder_status': 'O...,{'geocoded_waypoints': [{'geocoder_status': 'O...,{'geocoded_waypoints': [{'geocoder_status': 'O...,{'geocoded_waypoints': [{'geocoder_status': 'O...,...,49.225092,124.727992,36.724566,220.931639,69.955379,100.408401,47.541766,106.599806,61.359118,124.838121
1,Gateway - Green Valley Ranch,208600.0,1951.0,29201,23.98,Northeast,{'geocoded_waypoints': [{'geocoder_status': 'O...,{'geocoded_waypoints': [{'geocoder_status': 'O...,{'geocoded_waypoints': [{'geocoder_status': 'O...,{'geocoded_waypoints': [{'geocoder_status': 'O...,...,56.443832,200.352423,35.878963,205.355249,16.809006,102.592223,4.315476,149.179743,48.168008,132.352911
2,Hampden,297900.0,2138.0,17547,66.96,Southeast,{'geocoded_waypoints': [{'geocoder_status': 'O...,{'geocoded_waypoints': [{'geocoder_status': 'O...,{'geocoded_waypoints': [{'geocoder_status': 'O...,{'geocoded_waypoints': [{'geocoder_status': 'O...,...,9.702660,107.560628,18.167457,108.288770,47.749900,106.280323,51.106026,59.111560,37.639157,99.242317
3,Westwood,152600.0,1675.0,15486,11.39,West,{'geocoded_waypoints': [{'geocoder_status': 'O...,{'geocoded_waypoints': [{'geocoder_status': 'O...,{'geocoded_waypoints': [{'geocoder_status': 'O...,{'geocoded_waypoints': [{'geocoder_status': 'O...,...,88.411669,87.311828,75.334324,93.177966,39.914974,101.417961,39.929015,94.229550,40.214573,110.209966
4,Capitol Hill,197600.0,1534.0,14708,78.56,Central,{'geocoded_waypoints': [{'geocoder_status': 'O...,{'geocoded_waypoints': [{'geocoder_status': 'O...,{'geocoded_waypoints': [{'geocoder_status': 'O...,{'geocoded_waypoints': [{'geocoder_status': 'O...,...,52.433281,84.912461,38.058552,105.970982,36.316533,46.648238,39.241071,43.443411,28.317762,92.355886


In [3]:
#Denver.gov has a shapefile of neighborhoods - convert it to GeocodeJSON so gmaps
#can import it by using mapshaper.org
#import the Geocode JSON

with open('raw_data\\denver_neighborhood_outlines.json') as f:
    denver_neighborhood_geometry = json.load(f)

In [4]:
denver_neighborhood_geometry.get('features')[3].get('properties').get('NBHD_NAME')

'Washington Park'

In [5]:
#make the opacity of the neighborhood fill higher for higher price neighborhoods

#normalize the House Value to a list between 0.1 and 1.0 for opacity (minimum 
#at 0.1 so at least some shading and maximum at 1.0 so full opacity)
house_max = df['House Value'].max()
house_min = df['House Value'].min()
house_range = house_max - house_min
normalized_house_value = df['House Value'].map(lambda x: 
                                0.05 + (x-house_min)/house_range*0.95)

#create a dictionary of Neighborhood: House Value from df  and normalized list
neighborhood_house_price_dict = dict(zip(df['Neighborhood'], 
                                         normalized_house_value))

#cycle through the geojson in its order to create the list of opacity values
opacity_house_value = []
for feature in denver_neighborhood_geometry.get('features'):
    neighborhood = feature.get('properties').get('NBHD_NAME')
    opacity_house_value.append(neighborhood_house_price_dict.get(neighborhood))

#overlay the Denver Neighborhoods on gmaps and use opacity as defined
fig = gmaps.figure()

denver_neighborhood_layer = gmaps.geojson_layer(
                                denver_neighborhood_geometry,
                                fill_color='red',
                                fill_opacity=opacity_house_value
                                )

fig.add_layer(denver_neighborhood_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
#get lat/long of each neighborhood center so can place markers with information

# #Get Google Maps Geocode_JSON outlines of each Denver Neighborhood, then 
# #can use gmaps API to graph the neighborhoods on a data layer

# #https://developers.google.com/maps/documentation/geocoding/intro
# #https://maps.googleapis.com/maps/api/geocode/outputFormat?parameters
# #def get_neighborhood_geocode_json_boundaries(neighborhood_name):
# base_url = 'https://maps.googleapis.com/maps/api/geocode/json?'
# parameters = {'address': 'Montbello Neighborhood, Denver, CO', 
#               'key': google_maps_api_key
#               }

# geocode_json = requests.get(base_url, parameters)
# geocode = geocode_json.json()
# geocode

In [ ]:
#overlay the Denver Neighborhoods on gmaps
fig = gmaps.figure()

denver_neighborhood_layer = gmaps.geojson_layer(
                                denver_neighborhood_geometry,
                                fill_color='gray',
                                fill_opacity=0.2
                                )

fig.add_layer(denver_neighborhood_layer)
fig

In [ ]:
gmaps.geojson_layer?

In [79]:
###too many errors calculating average lat/long due to weird multi-polygon shapes and
###geocode-json encoding

# #get roughly the middle lat/long point of each neighborhood so create a marker
# #layer with the names on gmaps
# #note that geocode json flips to long/lat so need to account for that
# from collections import defaultdict 

# neighborhood_center = defaultdict(list)

# for feature in denver_neighborhood_geometry.get('features'):
#     neighborhood = feature.get('properties').get('NBHD_NAME')
#     print(neighborhood)
#     #some shapes are contiguous polygon and one list, others are MultiPolygon
#     #which gives multiple lists for each discontiguous polygon
#     if feature.get('geometry').get('type') == 'Polygon':
#         coordinates_neighborhood = feature.get('geometry').get('coordinates')[0]
#     elif feature.get('geometry').get('type') == 'MultiPolygon':
#         coordinates_neighborhood = []
#         for coord in feature.get('geometry').get('coordinates')[0]:
#             coordinates_neighborhood += coord
#     else: print('error- not Polygon or MultiPolygon Boundary')
#     longs=[]
#     lats=[]
#     for long_lat in coordinates_neighborhood:
#         longs.append(long_lat[0]) #append each longitude from a lat/long pair to a list of longs
#         lats.append(long_lat[1]) #append each longitude from a lat/long pair to a list of longs
#     center_lat = (min(lats) + max(lats))/2 #find average of min and max lat to get center
#     center_long = (min(longs) + max(longs))/2 #find average of min and max lat to get center
#     neighborhood_center[neighborhood]=[center_lat, center_long]
    
# neighborhood_center

In [ ]:
# #create a google map of the locations to see how spread out the random cities really are and to 
# #make sure it really covers various lat,longs
# fig3 = gmaps.figure()

# markers = gmaps.marker_layer(locations=df_random_cities3['Lat, Long'], 
#                              info_box_content=df_random_cities3['Country']) #click will give country code

# fig3.add_layer(markers)
# fig3